In [1]:
from Utils.FS import file
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
from scipy.sparse import coo_matrix, dok_matrix
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from wordcloud import WordCloud
from nltk.corpus import stopwords
import tensorflow as tf
import math

In [2]:
np.random.seed(1234)

In [3]:
sents = brown.sents()
sents = [[token.lower() for token in sent] for sent in sents]
words = brown.words()
words = [word.lower() for word in words]

In [4]:
print("Number of tokens: {}".format(len(words)))
print("Number of sentences: {}".format(len(sents)))
print("Longest sentences length: {}".format(max([len(sent) for sent in sents])))

Number of tokens: 1161192
Number of sentences: 57340
Longest sentences length: 180


In [5]:
def words2dicts(words):
    f_dict = {}
    index = 0
    for word in words:
        if not word in f_dict.keys():
            f_dict[word] = index
            index = index + 1
        
    return f_dict, { v:k for k, v in f_dict.items()}

In [6]:
words_dict, inv_words_dict = words2dicts(words)
words_size = len(words_dict)
print("Number of unique tokens: {}".format(words_size))

Number of unique tokens: 49815


In [7]:
def singleSideWindow(sents, words_dict, window_size, reverse = False):
    window = []
    row = []
    col = []
    data = []
        
    for sent in reversed(sents) if reverse else sents:
        for word in reversed(sent) if reverse else sent:
            for w in window:
                if w == word:
                    continue
                row.append(words_dict[word])
                col.append(words_dict[w])
                data.append(1)
            if len(window) == window_size:
                window.pop(0)
            window.append(word)
    return coo_matrix((data, (row, col)), shape=(len(words_dict), len(words_dict)), dtype='float64')
    

def sents2wordContextMatrix(sents, words_dict, window_size = 5):
    m = coo_matrix((words_size, words_size), 'float64')
    
    print('Doing forward pass...')
    m += singleSideWindow(sents, words_dict, window_size)
    
    print('Doing backward pass...')
    m += singleSideWindow(sents, words_dict, window_size, True)
    
    return m

def sents2wordCoocurrenceMatrix(sents, words_dict, window_size = 10):
    #don't really care edge cases....

    window = []
    row = []
    col = []
    data = []
    for sent in sents:
        for word in sent:
            for i in range(len(window)- 1):
                for j in range(i+1, len(window)):
                    row += [words_dict[window[i]], words_dict[window[j]]]
                    col += [words_dict[window[j]], words_dict[window[i]]]
                    data += [1, 1]
            if len(window) == window_size:
                window.pop(0)
            window.append(word)
    print('Preparing sparse matrix...')
    print('Length of data: {}'.format(len(data)))
    return coo_matrix((data, (row,col)), shape=(words_size, words_size), dtype='float64').multiply(1/len(data))


In [8]:
def singleSideWindowGenerator(c, w, sents, words_dict, window_size = 5, batch_size = 32, reverse = False):
    window = ['--' for i in range(window_size)]
    for sent in reversed(sents) if reverse else sents:
        for word in reversed(sent) if reverse else sent:
            for context in window:
                c.append(words_dict[context])
                w.append([words_dict[word]])
                if(len(c) == batch_size):
                    yield c, w
                    c = []
                    w = []
            window.pop(0)
            window.append(word)

def sents2batchGenerator(sents, words_dict, window_size = 5, batch_size=32):
    c = []
    w = []

    while(True):
        window = ['--' for i in range(window_size)]
        for sent in sents:
            for word in sent:
                for context in window:
                    c.append([words_dict[context]])
                    w.append(words_dict[word])
                    if(len(c) == batch_size):
                        yield w, c
                        c = []
                        w = []
                window.pop(0)
                window.append(word)

        window = ['--' for i in range(window_size)]
        for sent in reversed(sents):
            for word in reversed(sent):
                for context in window:
                    c.append([words_dict[context]])
                    w.append(words_dict[word])
                    if(len(c) == batch_size):
                        yield w, c
                        c = []
                        w = []
                window.pop(0)
                window.append(word)

    

In [9]:
def sents2freq(sents):
    freq = {}
    for sent in sents:
        for word in sent:
            if word in freq.keys():
                freq[word] += 1
            else:
                freq[word] = 1
    return freq

words_freq = sents2freq(sents)

In [10]:
WINDOW_SIZE = 2
BATCH_SIZE = 256
generator = sents2batchGenerator(sents, words_dict, window_size = WINDOW_SIZE, batch_size=BATCH_SIZE)

In [11]:
DIMENSION = 50
VOCABULAY_SIZE = len(words_dict)
NEGATIVE_SAMPLE = 64

graph = tf.Graph()

with graph.as_default():
    
    inputs = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
    labels = tf.placeholder(tf.int32, shape=[BATCH_SIZE, 1])

    with tf.device('/cpu:0'):
    
        embeddings = tf.Variable(
            tf.random_uniform([VOCABULAY_SIZE, DIMENSION], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, inputs, max_norm=1)

        nce_weights = tf.Variable(
            tf.truncated_normal([VOCABULAY_SIZE, DIMENSION],
                                stddev=1.0 / math.sqrt(DIMENSION)))

        nce_biases = tf.Variable(tf.zeros([VOCABULAY_SIZE]))

        loss = tf.reduce_mean(
          tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=labels,
                     inputs=embed,
                     num_sampled=NEGATIVE_SAMPLE,
                     num_classes=VOCABULAY_SIZE))

        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        init = tf.global_variables_initializer()
        word2VecSaver = tf.train.Saver({'Words2Vec': embeddings})

In [12]:
num_steps = 200000
MODEL = './model/brown-Words2Vec.ckpt'

with tf.Session(graph=graph) as session:
    init.run()
      
    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = next(generator)
        feed_dict = {inputs: batch_inputs, labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
    save_path = word2VecSaver.save(session, MODEL)
    final_embeddings = embeddings.eval()

Average loss at step  2000 :  155.518452305
Average loss at step  4000 :  101.466267084
Average loss at step  6000 :  76.9836863918
Average loss at step  8000 :  61.1315641532
Average loss at step  10000 :  49.8346918235
Average loss at step  12000 :  41.764257215
Average loss at step  14000 :  36.1083338292
Average loss at step  16000 :  30.2127132185
Average loss at step  18000 :  27.1740004787
Average loss at step  20000 :  23.6130024529
Average loss at step  22000 :  19.8649007888
Average loss at step  24000 :  17.3172629857
Average loss at step  26000 :  15.3189559948
Average loss at step  28000 :  13.5107321647
Average loss at step  30000 :  11.9894891266
Average loss at step  32000 :  11.0802330248
Average loss at step  34000 :  10.1879241896
Average loss at step  36000 :  9.71897320855
Average loss at step  38000 :  9.15163761973
Average loss at step  40000 :  8.13642629135
Average loss at step  42000 :  7.49171339059
Average loss at step  44000 :  6.84591049397
Average loss at

In [13]:

from sklearn.preprocessing import normalize

normalize(final_embeddings, norm='l2', axis=1, copy=False)
print(final_embeddings.shape)

(49815, 50)


In [14]:
sum((final_embeddings[2] ** 2))

0.99999999633581638

In [15]:
words_vec = {}
for i in range(final_embeddings.shape[0]):
    words_vec[inv_words_dict[i]] = final_embeddings[i]

In [16]:
def plotData(vocabs, X, Y):
    plt.clf()
    plt.figure(figsize=(36, 36))
    plt.scatter(X, Y)
    plt.axis([min(X), max(X), min(Y), max(Y)])
    for label, x, y in zip(vocabs, X, Y):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.show()    

In [17]:
def plot(vocabs, words_vec):
    X = [words_vec[vocab][0] for vocab in vocabs]
    Y = [words_vec[vocab][1] for vocab in vocabs]
    plotData(vocabs, X, Y)

In [18]:
def plotTSNE(vocabs, vectors):
    tsne = TSNE(perplexity=30, n_components=2, n_iter=5000, random_state = 7890, method='exact')
    #np.set_printoptions(suppress=True)
    data = np.array([vectors[vocab] for vocab in vocabs])    
    DATA = tsne.fit_transform(data)
    X = DATA[:, 0]
    Y = DATA[:, 1]
    
    plotData(vocabs, X, Y)

In [19]:
from DataLoader import GloVe

glove = GloVe.load2('./data/GloVe/glove.6B.50d.txt')

Start: Loading Glove Model
End: Loaded 400000 rows.


In [20]:
vocabs = ['man', 'woman', 'king', 'queen', 'male', 'female', 'boy', 'girl']
np.random.seed(1234)

random_vocabs = []
for i in np.random.randint(0, len(words_dict), 2000):
    if inv_words_dict[i] in glove.keys():
        random_vocabs.append(inv_words_dict[i])
        
print(len(random_vocabs))

1616


In [21]:
#plotTSNE(random_vocabs, words_vec)

In [22]:
#plotTSNE(random_vocabs, glove)

In [23]:
def cloestWord(word, words_vec, count = 10, method=None):
    if method == 'cos':
        dist = np.array([ sum(words_vec[word] * words_vec[key]) for key in words_vec.keys()])
        top_ten = dist.argsort()[::-1][:10]
    else:
        dist = np.array([ sum(np.square(np.array(words_vec[word]) - np.array(words_vec[key]))) for key in words_vec.keys()])
        top_ten = dist.argsort()[:10]
    return [list(words_vec.keys())[i] for i in top_ten]

In [24]:
print(cloestWord('man', words_vec, method='cos'))
print(cloestWord('man', glove))

['man', 'woman', 'child', 'person', 'married', 'mother', 'established', 'girl', 'met', 'provided']
['man', 'woman', 'another', 'boy', 'one', 'old', 'turned', 'whose', 'himself', 'who']


In [25]:
print(cloestWord('woman', words_vec, method='cos'))
print(cloestWord('woman', glove))

['woman', 'girl', 'child', 'boy', 'drink', 'person', 'job', 'friend', 'firm', 'decision']
['woman', 'girl', 'man', 'mother', 'boy', 'her', 'she', 'herself', 'victim', 'child']


In [26]:
print(cloestWord('however', words_vec, method='cos'))
print(cloestWord('however', glove))

['however', 'therefore', 'moreover', 'indeed', 'finally', 'especially', 'etc.', 'i.e.', 'nevertheless', 'besides']
['however', 'although', 'though', 'as', 'both', 'latter', 'also', '.', 'same', 'fact']


In [27]:
print(cloestWord('his', words_vec, method='cos'))
print(cloestWord('his', glove))

['his', 'their', 'its', 'her', 'my', 'our', 'the', 'every', 'your', 'a']
['his', 'he', 'himself', 'him', 'took', 'came', 'her', 'when', 'having', 'she']


In [28]:
print(cloestWord('zero', words_vec, method='cos'))
print(cloestWord('zero', glove))

['zero', 'gathering', 'dominant', 'jesus', 'healthy', 'rector', 'turning', 'thrown', 'forests', 'somewhere']
['zero', 'mean', 'assuming', 'hence', 'actual', 'approaching', 'comparable', 'measured', 'limit', 'comparison']


In [29]:
print(cloestWord('one', words_vec, method='cos'))
print(cloestWord('one', glove))

['one', 'knowledge', 'cost', 'importance', 'members', 'use', 'study', 'features', 'lack', 'value']
['one', 'another', 'only', 'same', '.', 'as', 'well', 'but', 'with', 'making']


In [30]:
print(cloestWord('two', words_vec, method='cos'))
print(cloestWord('two', glove))

['two', 'three', 'four', 'several', 'five', 'six', 'other', 'seven', 'various', 'many']
['two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'with', 'several']
